# Technical Test - KSG - Guido Bonomini
## Creating a Weather Agent using a TemperatureTool and a CoordinatesTool

### Step-by-Step Overview
This notebook consists of four main sections:

- Prep work for creating the ReaderTool:
    - Upload the Constitution.zip into the Blob Container
    - Create indexes for the Constitution book
    - Test indexes

- Setting up two MCP Servers (Flask) each providing a unique functionality:

    - A Reader Server returns the information on the uploaded pdf files.

    - A Infoleg Server that returns the Infoleg (Argentina Ministry of Justice) website.

- Creating LangChain Tools: These tools allow our AI agent to interact seamlessly with the MCP servers we just built.

- Building the AI Agent with LangChain: Using LangChain's React agent framework, we'll set up an intelligent AI agent to leverage our custom tools.

- Testing the Weather Agent: Finally, we'll run queries through the agent and observe its responses.

### Step 1. Setup code

In [ ]:
import os
import io
import json
import time
import requests
import random
import uuid
import shutil
import zipfile
from collections import OrderedDict
import urllib.request
from tqdm import tqdm
from bs4 import BeautifulSoup

from typing import List

from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
from langchain_core.runnables import ConfigurableField
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from operator import itemgetter

from common.utils import upload_file_to_blob, extract_zip_file, upload_directory_to_blob
from common.utils import parse_pdf, read_pdf_files
from common.prompts import DOCSEARCH_PROMPT_TEXT
from common.utils import CustomAzureSearchRetriever


from IPython.display import Markdown, HTML, display  

from dotenv import load_dotenv
load_dotenv("credentials.env")

def printmd(string):
    display(Markdown(string))

In [ ]:
# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]

#### Upload book to Blob Container

In [ ]:
%%time

# Define connection string and other parameters
BLOB_CONTAINER_NAME = "books"
BLOB_NAME = "constitution.zip"
LOCAL_FILE_PATH = "./data/" + BLOB_NAME  # Path to the local file you want to upload
upload_directory = "./data/temp_extract"  # Temporary directory to extract the zip file

# Extract the zip file
extract_zip_file(LOCAL_FILE_PATH, upload_directory)

# Upload the extracted files and folder structure
upload_directory_to_blob(upload_directory, BLOB_CONTAINER_NAME)

# Clean up: Optionally, you can remove the temp folder after uploading
shutil.rmtree(upload_directory)
print(f"Temp Folder: {upload_directory} removed")

#### Manual Document Cracking with Push to Vector-based Index

In [ ]:
# Dictionary to store the parsed data for each book
book_pages_map = dict()

# Open the zip file
with zipfile.ZipFile(LOCAL_FILE_PATH, 'r') as zip_ref:
    # Iterate over the PDF files inside the zip archive
    for file_info in zip_ref.infolist():
        if file_info.filename.endswith('.pdf'):
            book = file_info.filename
            
            print("Extracting Text from", book, "...")
            
            # Read the PDF file directly into memory (as a binary stream)
            with zip_ref.open(file_info) as file:
                file_stream = io.BytesIO(file.read())  # Convert file to BytesIO for in-memory file handling

                # Capture the start time
                start_time = time.time()

                # Parse the PDF (you would use your actual parse_pdf function here)
                book_map = parse_pdf(file_stream, form_recognizer=False, verbose=True)
                book_pages_map[book] = book_map
                
                # Capture the end time and calculate the elapsed time
                end_time = time.time()
                elapsed_time = end_time - start_time

                print(f"Parsing took: {elapsed_time:.6f} seconds")
                print(f"{book} contained {len(book_map)} pages\n")

#### Create the Laws Index

In [ ]:
batch_size = 75
embedder = AzureOpenAIEmbeddings(deployment=os.environ["EMBEDDING_DEPLOYMENT_NAME"], chunk_size=batch_size, 
                                 max_retries=2, 
                                 retry_min_seconds= 60,
                                 retry_max_seconds= 70)

In [ ]:
laws_index_name = "srch-index-laws"

In [ ]:
### Create Azure Search Vector-based Index
# Setup the Payloads header
headers = {'Content-Type': 'application/json','api-key': os.environ['AZURE_SEARCH_KEY']}
params = {'api-version': os.environ['AZURE_SEARCH_API_VERSION']}

In [ ]:
index_payload = {
    "name": laws_index_name,
    "vectorSearch": {
        "algorithms": [  # We are showing here 3 types of search algorithms configurations that you can do
             {
                 "name": "my-hnsw-config-1",
                 "kind": "hnsw",
                 "hnswParameters": {
                     "m": 4,
                     "efConstruction": 400,
                     "efSearch": 500,
                     "metric": "cosine"
                 }
             },
             {
                 "name": "my-hnsw-config-2",
                 "kind": "hnsw",
                 "hnswParameters": {
                     "m": 8,
                     "efConstruction": 800,
                     "efSearch": 800,
                     "metric": "cosine"
                 }
             },
             {
                 "name": "my-eknn-config",
                 "kind": "exhaustiveKnn",
                 "exhaustiveKnnParameters": {
                     "metric": "cosine"
                 }
             }
        ],
        "vectorizers": [
            {
                "name": "openai",
                "kind": "azureOpenAI",
                "azureOpenAIParameters":
                {
                    "resourceUri" : os.environ['AZURE_OPENAI_ENDPOINT'],
                    "apiKey" : os.environ['AZURE_OPENAI_API_KEY'],
                    "deploymentId" : os.environ['EMBEDDING_DEPLOYMENT_NAME'],
                    "modelName" : os.environ['EMBEDDING_DEPLOYMENT_NAME']
                }
            }
        ],
        "profiles": [  # profiles is the diferent kind of combinations of algos and vectorizers
            {
             "name": "my-vector-profile-1",
             "algorithm": "my-hnsw-config-1",
             "vectorizer":"openai"
            },
            {
             "name": "my-vector-profile-2",
             "algorithm": "my-hnsw-config-2",
             "vectorizer":"openai"
            },
            {
             "name": "my-vector-profile-3",
             "algorithm": "my-eknn-config",
             "vectorizer":"openai"
            }
        ]
    },
    "semantic": {
        "configurations": [
            {
                "name": "my-semantic-config",
                "prioritizedFields": {
                    "titleField": {
                        "fieldName": "title"
                    },
                    "prioritizedContentFields": [
                        {
                            "fieldName": "chunk"
                        }
                    ],
                    "prioritizedKeywordsFields": []
                }
            }
        ]
    },
    "fields": [
        {"name": "id", "type": "Edm.String", "key": "true", "filterable": "true" },
        {"name": "title","type": "Edm.String","searchable": "true","retrievable": "true"},
        {"name": "chunk","type": "Edm.String","searchable": "true","retrievable": "true"},
        {"name": "name", "type": "Edm.String", "searchable": "true", "retrievable": "true", "sortable": "false", "filterable": "false", "facetable": "false"},
        {"name": "location", "type": "Edm.String", "searchable": "false", "retrievable": "true", "sortable": "false", "filterable": "false", "facetable": "false"},
        {"name": "page_num","type": "Edm.Int32","searchable": "false","retrievable": "true"},
        {
            "name": "chunkVector",
            "type": "Collection(Edm.Single)",
            "dimensions": 3072,
            "vectorSearchProfile": "my-vector-profile-3", # we picked profile 3 to show that this index uses eKNN vs HNSW (on prior notebooks)
            "searchable": "true",
            "retrievable": "true",
            "filterable": "false",
            "sortable": "false",
            "facetable": "false"
        }
        
    ],
}

r = requests.put(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexes/" + laws_index_name,
                 data=json.dumps(index_payload), headers=headers, params=params)
print(r.status_code)
print(r.ok)

In [ ]:
# Function to process a batch of pages
def process_batch(bookname, pages):
    try:
        contents = [page[2] for page in pages]
        chunk_vectors = embedder.embed_documents(contents)
        
        upload_payload = {"value": []}
        for i, page in enumerate(pages):
            page_num = page[0] + 1
            content = page[2]
            book_url = os.environ['BASE_CONTAINER_URL'] + bookname
            
            payload = {
                "@search.action": "upload",
                "id": str(uuid.uuid5(uuid.NAMESPACE_DNS, f"{bookname}{page_num}")),
                "title": f"{bookname}_page_{str(page_num)}",
                "chunk": content,
                "chunkVector": chunk_vectors[i],
                "name": bookname,
                "location": book_url,
                "page_num": page_num
            }
            upload_payload["value"].append(payload)
        
        r = requests.post(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexes/" + laws_index_name + "/docs/index",
                          data=json.dumps(upload_payload), headers=headers, params=params)
        if r.status_code != 200:
            print(f"Failed to upload batch of pages from {bookname}: {r.status_code}")
            print(r.text)
    except Exception as e:
        print(f"Exception processing batch of pages from {bookname}: {e}")
        time.sleep(10)  # Wait before retrying
        process_batch(bookname, pages)  # Retry the same batch

In [ ]:
%%time
for bookname, bookmap in book_pages_map.items():
        print("Uploading chunks from", bookname)
        # Split bookmap into chunks of size chunk_size
        for i in tqdm(range(0, len(bookmap), batch_size)):
            batch = bookmap[i:i + batch_size]
            process_batch(bookname, batch)

#### Query the Index

In [ ]:
QUESTION = "Cuál es el primer artículo de la constitución?"

In [ ]:
indexes = [laws_index_name]
k=50 # in this index k corresponds to the top pages as well

In [ ]:
retriever = CustomAzureSearchRetriever(indexes=[laws_index_name], topK=k, reranker_threshold=1)

In [ ]:
COMPLETION_TOKENS = 2500
llm = AzureChatOpenAI(deployment_name=os.environ["GPT4oMINI_DEPLOYMENT_NAME"], temperature=0.5, max_tokens=COMPLETION_TOKENS).configurable_alternatives(
    ConfigurableField(id="model"),
    default_key="gpt4omini",
    gpt4o=AzureChatOpenAI(deployment_name=os.environ["GPT4o_DEPLOYMENT_NAME"], temperature=0, max_tokens=COMPLETION_TOKENS),
)

In [ ]:
DOCSEARCH_PROMPT = ChatPromptTemplate.from_messages(
    [
        ("system", DOCSEARCH_PROMPT_TEXT + "\n\nCONTEXT:\n{context}\n\n"),
        ("human", "{question}"),
    ]
)

In [ ]:
chain = (
    {
        "context": itemgetter("question") | retriever, # Passes the question to the retriever and the results are assign to context
        "question": itemgetter("question")
    }
    | DOCSEARCH_PROMPT  # Passes the 4 variables above to the prompt template
    | llm   # Passes the finished prompt to the LLM
    | StrOutputParser()  # converts the output (Runnable object) to the desired output (string)
)

In [ ]:
for chunk in chain.with_config(configurable={"model": "gpt4o"}).stream(
    {"question": QUESTION, "language": "Spanish"}):
    print(chunk, end="", flush=True)

### Step 2: Creating LangChain Tools

To enable the AI agent to interact with our MCP servers, we must create LangChain tools. These tools act as intermediaries, sending HTTP requests to our Flask endpoints and fetching the responses.

In [ ]:
from concurrent.futures import ThreadPoolExecutor
from typing import Type, Optional
from pydantic import BaseModel
from langchain.tools import BaseTool, StructuredTool
import requests
import asyncio
from langchain_core.callbacks import AsyncCallbackManagerForToolRun,CallbackManagerForToolRun

import requests
from bs4 import BeautifulSoup
import json
import asyncio
from concurrent.futures import ThreadPoolExecutor
from typing import Type
from pydantic import BaseModel

class ReaderSearchInput(BaseModel):
    query: str
    

class ReaderTool(BaseTool):
    name: str = "ReaderTool"
    description: str = "Use this tool to search Argentine legal documents from Infoleg based on a query."
    args_schema: Type[BaseModel] = ReaderSearchInput
    
    indexes: List[str] = [laws_index_name]
    k: int = 50
    reranker_th: float = 1

    def _run(self, query: str,  return_direct = False, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:

        retriever = CustomAzureSearchRetriever(indexes=self.indexes, topK=self.k, reranker_threshold=self.reranker_th, 
                                               callback_manager=self.callbacks)
        results = retriever.invoke(input=query)
        
        return results

    async def _arun(self, query: str, return_direct = False, run_manager: Optional[AsyncCallbackManagerForToolRun] = None) -> str:
        """Use the tool asynchronously."""
        
        retriever = CustomAzureSearchRetriever(indexes=self.indexes, topK=self.k, reranker_threshold=self.reranker_th, 
                                               callback_manager=self.callbacks)
        loop = asyncio.get_event_loop()
        results = await loop.run_in_executor(ThreadPoolExecutor(), retriever.invoke, query)
        
        return results


class CriminalCodeTool(BaseTool):
   """Tool for fetching and returning the text content, image URLs, and links of the Argentina government web page, capped at a maximum number of words."""
   name: str = "CriminalCodeTool"
   description: str = "Use this tool to extract text or specific articles from the Código Penal Argentino hosted on Infoleg."
   url: str = "https://www.argentina.gob.ar/normativa/nacional/ley-11179-16546/texto"
   max_words: int = 10000
   def _run(self) -> str:
       """Synchronously fetches the Argentina government web page and returns its text content, image URLs, and links capped at max_words."""
       try:
           headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:90.0) Gecko/20100101 Firefox/90.0'}
           response = requests.get(self.url, headers=headers, timeout=10)
           response.raise_for_status()
           soup = BeautifulSoup(response.content, 'html.parser')
           # Extract text content
           text_content = soup.get_text()
           words = text_content.split()
           capped_words = words[:self.max_words] if len(words) > self.max_words else words
           text_result = ' '.join(capped_words)
           
           result = {
               "text_content": text_result
           }
           
           return json.dumps(result)
       
       except Exception as e:
           return f"Error fetching or parsing Infoleg page: {str(e)}"
   
   async def _arun(self) -> str:
       """Asynchronously fetches a webpage and returns its text content, image URLs, and links capped at max_words."""
       loop = asyncio.get_event_loop()
       try:
           result = await loop.run_in_executor(ThreadPoolExecutor(), lambda: self._run())
           return result
       except Exception as e:
           return json.dumps({"error": str(e)})
        

### Step 3: Building the AI Agent with LangChain

With our tools ready, we now integrate them into an Agent. The agent is built using the React agent pattern, allowing it to dynamically decide when and how to use our custom MCP tools to answer queries.

In [ ]:
from langchain.chat_models import AzureChatOpenAI
from langgraph.prebuilt import create_react_agent

def create_law_agent(llm: AzureChatOpenAI, prompt:str, name: str):
    reader_tool = ReaderTool()
    criminal_code_tool = CriminalCodeTool()
    
    law_agent = create_react_agent(
        llm, 
        tools=[reader_tool, criminal_code_tool], 
        prompt=prompt,
        name=name
    )
    
    # Optional tagging for filtering or identification
    law_agent = law_agent.with_config(tags=[name])
    
    return law_agent


### Step 4: Testing the Law Agent

Let's now test our Law Agent to verify that it correctly fetches data from the MCP tools and responds intelligently.

In [ ]:
import os
import random
import json
import uuid
import requests
import logging
import functools
import operator
from pydantic import BaseModel
from typing import Annotated, Sequence, Literal
from typing_extensions import TypedDict

from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage, BaseMessage

from langgraph.graph import END, StateGraph, START
from langgraph.checkpoint.serde.jsonplus import JsonPlusSerializer


from common.cosmosdb_checkpointer import CosmosDBSaver, AsyncCosmosDBSaver

#custom libraries that we will use later in the app
from common.utils import (
    create_docsearch_agent,
    create_csvsearch_agent,
    create_sqlsearch_agent,
    create_websearch_agent,
    create_apisearch_agent,
    reduce_openapi_spec
)

from common.prompts import (
    CUSTOM_CHATBOT_PREFIX,
    DOCSEARCH_PROMPT_TEXT,
    CSV_AGENT_PROMPT_TEXT,
    MSSQL_AGENT_PROMPT_TEXT,
    WEBSEARCH_PROMPT_TEXT,
    APISEARCH_PROMPT_TEXT,
    LEGAL_PROMPT_TEXT
)

from dotenv import load_dotenv
load_dotenv("credentials.env")

from IPython.display import Image, Markdown, Audio, display 

from common.audio_utils import text_to_speech 

def play_audio(file_path):
    """Play an audio file in Jupyter Notebook."""
    display(Audio(file_path, autoplay=True))

def printmd(string):
    # Remove ```markdown and ``` from the text
    clean_content = re.sub(r'^```markdown\n', '', string)
    clean_content = re.sub(r'^```\n', '', clean_content)
    clean_content = re.sub(r'\n```$', '', clean_content)

    # Escape dollar signs to prevent LaTeX rendering
    clean_content = clean_content.replace('$', r'\$')
    display(Markdown(clean_content))

In [ ]:
COMPLETION_TOKENS = 5000

llm = AzureChatOpenAI(deployment_name=os.environ["GPT4o_DEPLOYMENT_NAME"], 
                      temperature=0, max_tokens=COMPLETION_TOKENS, 
                      streaming=True)


#### Creating all of the Specialized Agents to use with the Supervisor

#### **Law Agent**

In [ ]:
law_agent = create_law_agent(llm, prompt=CUSTOM_CHATBOT_PREFIX + LEGAL_PROMPT_TEXT,
                                     name="LawSearch")

#### **DocSearch Agent**

In [ ]:
indexes = ["srch-index-files", "srch-index-csv", "srch-index-books"]
docsearch_agent = create_docsearch_agent(llm,indexes,k=20,reranker_th=1.5,
                                         prompt=CUSTOM_CHATBOT_PREFIX + DOCSEARCH_PROMPT_TEXT,
                                         sas_token=os.environ['BLOB_SAS_TOKEN'],
                                         name="DocSearch"
                                        )

#### **CSVSearch Agent**

In [ ]:
file_url = "./data/all-states-history.csv"
csvsearch_agent = create_csvsearch_agent(llm,
                                         prompt=CUSTOM_CHATBOT_PREFIX + CSV_AGENT_PROMPT_TEXT.format(file_url=file_url),
                                         name="CSVSearch")

#### **SQLSearch Agent**

In [ ]:
sqlsearch_agent = create_sqlsearch_agent(llm, 
                                     prompt=CUSTOM_CHATBOT_PREFIX + MSSQL_AGENT_PROMPT_TEXT,
                                     name="SQLSearch")

#### **WebSearch Agent**

In [ ]:
websearch_agent = create_websearch_agent(llm, 
                                     prompt=CUSTOM_CHATBOT_PREFIX + WEBSEARCH_PROMPT_TEXT,
                                     name="WebSearch")

#### **APISearch Agent**

In [ ]:
api_file_path = "./data/openapi_kraken.json"
with open(api_file_path, 'r') as file:
    spec = json.load(file)
    
reduced_api_spec = reduce_openapi_spec(spec)

apisearch_agent = create_apisearch_agent(llm, 
                                     prompt=CUSTOM_CHATBOT_PREFIX + APISEARCH_PROMPT_TEXT.format(api_spec=reduced_api_spec),
                                     name="APISearch")

#### Helper Print and Audio Functions

Define functions to print the events and respond with Audio.

These are two different ways to print and stream the answers and events

In [ ]:
# Define a sync function to stream graph updates
def stream_graph_updates_sync(user_input: str, graph, config):
    last_agent_message = ""  # Will hold the latest AIMessage content

    for event in graph.stream({"messages": [("human", user_input)]}, config, stream_mode="updates"):
        # print(event)  # Print the raw event (keep this for debugging if needed)

        # Each event is a dict, e.g. {"WebSearchAgent": {...}} or {"supervisor": {"messages": [...]}}
        if isinstance(event, dict):
            for key, value in event.items():
                # If this is an agent or supervisor event, store the latest message content
                # (No change here—this correctly captures all updates, with the final one being the supervisor's response)
                if isinstance(value, dict) and "messages" in value:
                    messages = value["messages"]
                    if messages:
                        last_msg = messages[-1]
                        # Only update last_agent_message if it's an AIMessage with content
                        # (Added this check to avoid overwriting with intermediate ToolMessages or empty contents;
                        # ensures we prioritize AIMessages, which are the actual responses)
                        if hasattr(last_msg, 'content') and last_msg.content and not getattr(last_msg, 'tool_calls', None):
                            last_agent_message = last_msg.content

    # It triggers after all events, using the final last_agent_message (supervisor's response)
    if last_agent_message:
        print(last_agent_message)
        tts_audio_file = text_to_speech(last_agent_message)
        if tts_audio_file:
            play_audio(tts_audio_file)



# Define an async function to stream events async
async def stream_graph_updates_async(user_input: str, graph, config, exclude_tags_list=[] ):
    inputs = {"messages": [("human", user_input)]}
    complete_text = ""  # Store the full response text for TTS
    if config is None:
        config = {}

    async for event in graph.astream_events(inputs, config, exclude_tags=exclude_tags_list, version="v2"):
        # print(event)
        
        # Added: Reset complete_text on each chat model start from the 'agent' node
        # This ensures complete_text only holds the output from the most recent model call,
        # which will be the final supervisor response at the end of processing
        if (
            event["event"] == "on_chat_model_start"
            and event["metadata"].get("langgraph_node") == "agent"
        ):
            # print(event)
            complete_text = ""
        
        if (
            event["event"] == "on_chat_model_stream"  # Ensure the event is a chat stream event
            and event["metadata"].get("langgraph_node") == "agent"
        ):
            # Print the content of the chunk progressively
            chunk_text = event["data"]["chunk"].content
            print(chunk_text, end="", flush=True)
            complete_text += chunk_text  # Accumulate chunks of text

        if (
            event["event"] == "on_tool_start"  
            and event["metadata"].get("langgraph_node") == "tools"  # Ensure it's from the tools node
        ):
            print("\n--")
            print(f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}")
            print("--")
        if (
            event["event"] == "on_tool_end"  # Ensure the event is a chat stream event
            and event["metadata"].get("langgraph_node") == "tools"  # Ensure it's from the chatbot node
        ):
            print("\n--")
            print(f"Done tool: {event['name']}")
            print("--")
            
    # Moved TTS here, outside the loop (key fix: triggers after all events, using the final accumulated complete_text)
    # Removed the 'next' == 'FINISH' check, as it's not present in events and unnecessary
    if complete_text:
        tts_audio_file = text_to_speech(complete_text)
        if tts_audio_file:
            play_audio(tts_audio_file)
    

#### Create supervisor with `langgraph-supervisor`

To implement out multi-agent system, we will use [`create_supervisor`]from the prebuilt `langgraph-supervisor` library:


In [ ]:
from langgraph_supervisor import create_supervisor

supervisor = create_supervisor(
    model=llm,
    agents=[law_agent, docsearch_agent, csvsearch_agent, websearch_agent, apisearch_agent],
    prompt=(
        "You are a supervisor managing several agents:\n"
        "- a LawSearch agent. Assign tasks to this agent if the user states: @weathersearch.\n"
        "- a DocSearch agent. Assign tasks to this agent if the user states: @docsearch.\n"
        "- a CSVSearch agent. Assign tasks to this agent if the user states: @csvsearch.\n"
        "- a WebSearch agent. Assign tasks to this agent if the user states: @websearch.\n"
        "- a ApiSearch agent. Assign tasks to this agent if the user states: @apisearch.\n"
        "Assign work to one agent at a time, do not call agents in parallel.\n"
        "Do not do any work yourself."
    ),
    add_handoff_back_messages=True,
    output_mode="full_history",
)

checkpointer_sync = CosmosDBSaver(
    endpoint=os.environ["AZURE_COSMOSDB_ENDPOINT"],
    key=os.environ["AZURE_COSMOSDB_KEY"],
    database_name=os.environ["AZURE_COSMOSDB_NAME"],
    container_name=os.environ["AZURE_COSMOSDB_CONTAINER_NAME"],
    serde=JsonPlusSerializer(),
)

# Manually initialize resources
checkpointer_sync.setup()

# Compile the synchronous graph after setup is complete
graph_sync = supervisor.compile(checkpointer=checkpointer_sync)

In [ ]:
import uuid
# Define a test thread_id to store in the persistent storage
config_sync = {"configurable": {"thread_id": str(uuid.uuid4())}}

display(Image(graph_sync.get_graph().draw_mermaid_png()))

### Run SYNC App

In [ ]:
# Run the synchronous agent
print("Running the synchronous agent:")
while True:
    user_input = input("User: ")
    if user_input.lower() in ["quit", "exit", "q"]:
        print("Goodbye!")
        break
    try:
        stream_graph_updates_sync(user_input, graph_sync, config_sync)
    except Exception as e:
        print(f"Error during synchronous update: {e}")

### Construct the ASYNC graph of our application

#### Let's talk to our Engine ASYNC chat bot now

In [ ]:
# We can as well avoid the .setup() call of the cosmosDB by using the with statement as below
async def run_async_agent():
    async with AsyncCosmosDBSaver(
        endpoint=os.environ["AZURE_COSMOSDB_ENDPOINT"],
        key=os.environ["AZURE_COSMOSDB_KEY"],
        database_name=os.environ["AZURE_COSMOSDB_NAME"],
        container_name=os.environ["AZURE_COSMOSDB_CONTAINER_NAME"],
        serde=JsonPlusSerializer(),
    ) as checkpointer_async:
        # Compile the asynchronous graph
        graph_async = supervisor.compile(checkpointer=checkpointer_async)
        # Define a test thread_id to store in the persistent storage
        config_async = {"configurable": {"thread_id": str(uuid.uuid4())}}



        print("\nRunning the asynchronous agent:")
        while True:
            user_input = input("User: ")
            if user_input.lower() in ["quit", "exit", "q"]:
                print("Goodbye!")
                break
            await stream_graph_updates_async(user_input, graph_async ,config_async, exclude_tags_list=["WeatherSearch"])

# Run the asynchronous agent
await run_async_agent()